In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
import swifter
import nltk

In [2]:
NUM_WORKERS = 8
TEST_SIZE = 0.2
LABEL_COLUMN_NAME = "oh_label"
TEXT_COLUMN_NAME = "Text"
DATASET_PATH = "../Data/ver1.csv"

In [3]:
df= pd.read_csv(DATASET_PATH, index_col=False)
df = df.dropna()
df.drop(["Unnamed: 0", "index"],axis=1, inplace = True)

In [4]:
y = pd.Categorical(df[LABEL_COLUMN_NAME]).codes
X = df[TEXT_COLUMN_NAME]

In [5]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2021-12-31 13:48:41,743 loading file C:\Users\yosef\.flair\models\sentiment-en-mix-distillbert_4.pt


In [14]:
def get_sentiment_score(text):
    s = flair.data.Sentence(text,use_tokenizer=True)
    flair_sentiment.predict(s)
    if len(s.labels) == 0:
        return 0.5
    return s.labels[0].score
a = X[:100].swifter.apply(get_sentiment_score)

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

0     0.996872
1     0.970882
2     0.999994
3     0.997631
4     0.999477
        ...   
95    0.999943
96    0.995348
97    0.999686
98    0.999545
99    0.996615
Name: Text, Length: 100, dtype: float64

In [21]:
y[a.argmax()]

1

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X.values,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
model = Sequential()
model.add(Dense(60, activation="relu"))
model.add(Dense(30, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(4, activation="softmax"))
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), optimizer='adam',metrics=["accuracy"])
model.summary()

In [ ]:
weights = 1 - pd.Series(y).value_counts()/y.size
weights = weights.to_dict()
weights

In [ ]:
import tensorflow_addons as tfa
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
time_stopping_callback = tfa.callbacks.TimeStopping(seconds=60*60*8, verbose=1) 
history = model.fit(X_train,y_train,batch_size=200,epochs=1000,
                    validation_split=0.1,class_weight=weights,
                    use_multiprocessing=True,workers=8,callbacks=[callback,time_stopping_callback])